In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
from collections import Counter
import pandas as pd
import random

class GaugeConfigRecovery:
    def __init__(self, valid_configs):
        """
        valid_configs: list of bitstrings satisfying Gauss law (even parity)
        """
        self.valid = valid_configs
        self.freq = Counter()

    def repair_and_update(self, raw_counts):
        recovered = Counter()
        # accumulate raw frequencies
        for bits, cnt in raw_counts.items():
            self.freq[bits] += cnt
        # repair each sample to nearest valid config by Hamming distance
        for bits, cnt in raw_counts.items():
            best = min(self.valid, key=lambda v: sum(a!=b for a,b in zip(bits, v)))
            recovered[best] += cnt
        return recovered

# Build U(1) 2x2 plaquette gauge circuit
def build_u1_plaquette():
    qr = QuantumRegister(4, 'link')
    cr = ClassicalRegister(4, 'c')
    qc = QuantumCircuit(qr, cr)
    # initial superposition
    qc.h(qr)
    # plaquette rotation example
    theta = 0.3
    # implement Z⊗Z⊗Z⊗Z phase
    # map to Z basis, control chain
    qc.h(qr[2]); qc.h(qr[3])
    qc.cx(qr[0], qr[2]); qc.cx(qr[1], qr[2]); qc.cx(qr[2], qr[3])
    qc.rz(2*theta, qr[3])
    qc.cx(qr[2], qr[3]); qc.cx(qr[1], qr[2]); qc.cx(qr[0], qr[2])
    qc.h(qr[2]); qc.h(qr[3])
    # measure
    qc.measure(qr, cr)
    return qc

# Add noise flipping random links
def add_measurement_noise(raw_counts, flip_prob=0.1):
    noisy = Counter()
    for bits, cnt in raw_counts.items():
        for _ in range(cnt):
            b = ''.join(
                bit if random.random()>flip_prob else ('1' if bit=='0' else '0')
                for bit in bits
            )
            noisy[b] += 1
    return noisy

# generate valid gauge-invariant configs: even parity
def generate_valid_configs():
    valid = []
    for i in range(16):
        bits = format(i, '04b')
        if bits.count('1') % 2 == 0:
            valid.append(bits)
    return valid

def main():
    # 1) build gauge circuit
    qc = build_u1_plaquette()
    service = QiskitRuntimeService()
    backend = service.backend(name="ibm_aachen")
    qc_t = transpile(qc, backend=backend, optimization_level=3)
    with Session(backend=backend) as session:
        sampler = Sampler(backend)
        job = sampler.run([qc_t], shots=2048)
        raw_counts = job.result()[0].join_data().get_counts()

    # 2) add measurement noise
    noisy_counts = add_measurement_noise(raw_counts, flip_prob=0.1)

    # 3) dynamic config recovery
    valid = generate_valid_configs()
    dcr = GaugeConfigRecovery(valid)
    recovered = dcr.repair_and_update(noisy_counts)

    # 4) output results
    df = pd.DataFrame([{'config':b, 'raw':noisy_counts.get(b,0), 'recovered':recovered.get(b,0)}
                       for b in sorted(set(noisy_counts)|set(recovered))])
    print(df.to_string(index=False))

if __name__=='__main__':
    main()


base_primitive._get_mode_service_backend:WARNING:2025-07-01 02:31:08,328: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.


config  raw  recovered
  0000  103        621
  0001  138          0
  0010  140          0
  0011  131        397
  0100  111          0
  0101  115        231
  0110  123        237
  0111  135          0
  1000  129          0
  1001  138        138
  1010  140        140
  1011  131          0
  1100  138        138
  1101  116          0
  1110  114          0
  1111  146        146
